### 📄 Documentação: Extrator de Composição IBRX-100 (API B3)

### 1. Visão Geral

Este script é um componente de **Extração (E)** dentro do pipeline ETL do projeto Aurum. Sua função é conectar-se diretamente à API "oculta" da B3 (Brasil, Bolsa, Balcão) para obter a carteira teórica atualizada do índice **IBRX-100**.

Diferente de abordagens tradicionais que utilizam *web scraping* (Selenium/BeautifulSoup), este script utiliza engenharia reversa da chamada de API da B3, codificando parâmetros em Base64 para simular uma requisição legítima do navegador.



### 2. Dependências Técnicas

Para executar este script, o ambiente Python deve conter as seguintes bibliotecas:

* **pandas:** Manipulação e estruturação dos dados.
* **requests:** Realização de chamadas HTTP à API.
* **urllib3:** Gerenciamento de conexões e supressão de avisos SSL.
* **Bibliotecas Padrão:** `json`, `base64`, `logging`, `pathlib`, `time`.

### 3. Estrutura do Script

O script não utiliza Classes (POO), mas sim uma arquitetura funcional modular. Abaixo estão as descrições das principais funções:

### **fetch_ibrx100_from_b3_api()**

Esta é a função "core" do extrator.

1. **Prepara Parâmetros:** Define um dicionário JSON com o índice alvo (`IBXX` para IBRX-100) e paginação.
2. **Codificação Base64:** Transforma o JSON em uma string Base64, replicando o comportamento do frontend da B3.
3. **Requisição:** Envia um GET para o endpoint `indexProxy/indexCall/GetPortfolioDay`.
4. **Tratamento de Dados:**
    * Consome o JSON de resposta.
    * **Autocorreção de Colunas:** Verifica dinamicamente o nome da coluna de código do ativo (`codNeg`, `cod`, `acronym`), garantindo robustez caso a B3 altere a API.
    * **Normalização:** Remove espaços em branco e cria a coluna compatível com o Yahoo Finance (sufixo `.SA`).



### `save_data(df)`

Responsável pela persistência dos dados.

1. Verifica se o diretório `../data` existe; caso contrário, cria-o.
2. Salva o DataFrame em dois formatos:
    * **.CSV:** Para inspeção humana e compatibilidade simples.
    * **.PARQUET:** Para leitura de alta performance nos próximos passos do pipeline.

## 4. Dicionário de Dados (Output)

O script gera um dataset padronizado. A tabela abaixo descreve o esquema (schema) do arquivo gerado (`tickers_ibrx100_full`).

| Nome da Coluna | Tipo de Dado (Pandas) | Descrição | Exemplo |
| :--- | :--- | :--- | :--- |
| **`ticker`** | `object` (string) | O código de negociação oficial fornecido pela B3. | `PETR4` |
| **`participacao`** | `object` / `float` | (Opcional) A porcentagem de participação do ativo no índice, se retornada pela API. | `4,520` |
| **`Ticker_Yahoo`** | `object` (string) | O ticker normalizado com o sufixo `.SA`, pronto para consumo pela biblioteca `yfinance`. | `PETR4.SA` |

## 5. Como Executar

Execute o script diretamente via terminal:

```bash
python b3_api_extractor.py

In [ ]:
import pandas as pd
import requests
import base64
import json
import logging
from pathlib import Path
import time
import urllib3

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

OUTPUT_DIR = Path("..", "data") 
OUTPUT_FILENAME_CSV = "tickers_ibrx100_full.csv"
OUTPUT_FILENAME_PARQUET = "tickers_ibrx100_full.parquet"

def fetch_ibrx100_from_b3_api() -> pd.DataFrame:
    """
    Consome diretamente a API JSON da B3 para obter a composição do IBRX-100.
    """
    logger.info("Iniciando requisição à API da B3 (IndexProxy)...")
    
    try:
        params = {
            "language": "pt-br",
            "pageNumber": 1,
            "pageSize": 120, 
            "index": "IBXX", 
            "segment": "1"
        }
        
        params_json = json.dumps(params)
        params_b64 = base64.b64encode(params_json.encode("utf-8")).decode("utf-8")
        
        url = f"https://sistemaswebb3-listados.b3.com.br/indexProxy/indexCall/GetPortfolioDay/{params_b64}"
        
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }
        
        urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
        response = requests.get(url, headers=headers, timeout=15, verify=False)
        
        if response.status_code != 200:
            logger.error(f"Erro na requisição: Status {response.status_code}")
            return None
            
        data = response.json()
        results = data.get('results', [])
        
        if not results:
            logger.warning("JSON retornado pela B3 está vazio na chave 'results'.")
            return None
            
        logger.info(f"API retornou {len(results)} ativos.")
        
        df = pd.DataFrame(results)
        
        logger.info(f"Colunas encontradas no JSON: {df.columns.tolist()}")
        
        coluna_ticker = None
        possiveis_nomes = ['codNeg', 'cod', 'acronym', 'symbol', 'identifier']
        
        for col in possiveis_nomes:
            if col in df.columns:
                coluna_ticker = col
                logger.info(f"Coluna de ticker identificada como: '{col}'")
                break
        
        if not coluna_ticker:
            logger.error("Não foi possível identificar a coluna de Ticker no DataFrame.")
            logger.error(f"Colunas disponíveis: {df.columns.tolist()}")
            return None

        coluna_part = 'part' if 'part' in df.columns else None
        
        colunas_selecao = [coluna_ticker]
        if coluna_part:
            colunas_selecao.append(coluna_part)
            
        df_final = df[colunas_selecao].copy()
        
        rename_map = {coluna_ticker: 'ticker'}
        if coluna_part:
            rename_map[coluna_part] = 'participacao'
            
        df_final = df_final.rename(columns=rename_map)
        
        logger.info("Normalizando tickers (adicionando .SA)...")
        df_final['ticker'] = df_final['ticker'].str.strip()
        df_final['Ticker_Yahoo'] = df_final['ticker'].apply(lambda x: f"{x}.SA")
        
        return df_final

    except Exception as e:
        logger.error(f"Falha crítica no extrator da API B3: {e}")
        import traceback
        logger.error(traceback.format_exc())
        return None

def save_data(df: pd.DataFrame):
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    
    path_csv = OUTPUT_DIR / OUTPUT_FILENAME_CSV
    path_parquet = OUTPUT_DIR / OUTPUT_FILENAME_PARQUET
    
    df.to_csv(path_csv, index=False, encoding='utf-8-sig')
    df.to_parquet(path_parquet, index=False)
    
    logger.info(f"💾 Arquivos salvos:")
    logger.info(f"   -> {path_csv}")
    logger.info(f"   -> {path_parquet}")

if __name__ == "__main__":
    start_time = time.time()
    
    df_result = fetch_ibrx100_from_b3_api()
    
    if df_result is not None and not df_result.empty:
        print("\n--- Amostra do IBRX-100 (API B3) ---")
        display(df_result.head())
        save_data(df_result)
    else:
        logger.error("Não foi possível gerar a lista de tickers.")
        
    print(f"\nTempo total: {time.time() - start_time:.2f} segundos")

2025-12-12 02:00:32,745 - INFO - Iniciando requisição à API da B3 (IndexProxy)...
2025-12-12 02:00:33,723 - INFO - API retornou 97 ativos.
2025-12-12 02:00:33,733 - INFO - Colunas encontradas no JSON: ['segment', 'cod', 'asset', 'type', 'part', 'partAcum', 'theoricalQty']
2025-12-12 02:00:33,737 - INFO - Coluna de ticker identificada como: 'cod'
2025-12-12 02:00:33,744 - INFO - Normalizando tickers (adicionando .SA)...
2025-12-12 02:00:33,803 - INFO - 💾 Arquivos salvos:
2025-12-12 02:00:33,804 - INFO -    -> ..\data\tickers_ibrx100_full.csv
2025-12-12 02:00:33,804 - INFO -    -> ..\data\tickers_ibrx100_full.parquet



--- Amostra do IBRX-100 (API B3) ---
  ticker participacao Ticker_Yahoo
0  ALOS3        0,536     ALOS3.SA
1  ABEV3        2,359     ABEV3.SA
2  ANIM3        0,036     ANIM3.SA
3  ASAI3        0,424     ASAI3.SA
4  AURE3        0,157     AURE3.SA

Tempo total: 1.06 segundos


## 🔗 Mapeador Automático Ticker ↔ CNPJ (B3/CVM)

Este utilitário Python resolve o problema de desconexão de dados entre a **B3** (que opera via Tickers) e a **CVM** (que opera via CNPJ), criando um mapeamento confiável e automatizado sem intervenção manual.

## 🎯 O Problema (O "Elo Perdido")
Sistemas de análise financeira (como o **Aurum**) frequentemente precisam cruzar dados de cotação (B3) com dados fundamentalistas/cadastrais (CVM). No entanto:
* A **B3** fornece o Ticker (ex: `ABEV3`) mas raramente fornece o CNPJ ou a Razão Social completa na API pública.
* A **CVM** fornece o CNPJ e a Razão Social, mas não sabe qual é o Ticker associado.

Este script cria uma "ponte" inteligente utilizando o **Yahoo Finance** para descobrir o nome comercial e algoritmos de **Fuzzy Matching** para vinculá-lo ao CNPJ oficial.

## 🛠️ Como Funciona (Pipeline Lógico)



1.  **Extração B3:** O script consulta a API interna da B3 (`IndexProxy`) para obter a composição atualizada do índice **IBRX-100**.
2.  **Dados Oficiais CVM:** Baixa automaticamente o arquivo `cad_cia_aberta.csv` diretamente do portal de Dados Abertos da CVM.
3.  **Enriquecimento (A Ponte):** Para cada Ticker da B3, o script consulta o `yfinance` para descobrir o "Nome Longo" da empresa (ex: Converte `PETR4` → "Petróleo Brasileiro S.A. - Petrobras").
4.  **Matching Probabilístico:** Utiliza a biblioteca `rapidfuzz` para comparar o nome obtido no Yahoo com a Razão Social da CVM. Se a similaridade for alta (Score > 70), o vínculo é criado.

## 📋 Pré-requisitos

O script requer Python 3.8+ e as seguintes bibliotecas externas:

pip install pandas requests yfinance rapidfuzz urllib3

```mermaid 
graph TD
    %% Nós de Início e Fim
    Start([Início: main]) --> FetchB3
    End([Fim: Salvar CSV e Estatísticas])

    %% ETAPA 1: Aquisição de Dados
    subgraph "1. Aquisição de Dados"
        FetchB3[📡 Fetch API B3: IBRX-100] --> CheckB3{Sucesso?}
        CheckB3 -- Não --> Stop1([Encerrar])
        CheckB3 -- Sim --> FetchCVM

        FetchCVM[🏛️ Fetch Cadastro CVM Web] --> CheckCVM{Sucesso?}
        CheckCVM -- Não --> Stop2([Encerrar])
        CheckCVM -- Sim --> LoadLocal

        %% CORREÇÃO AQUI: Aspas adicionadas ao redor do texto
        LoadLocal["📂 Load Local Fundamentals<br/>(fundamentals_wide.csv)"] --> CheckLocal{Existe?}
        
        CheckLocal -- Sim --> DataReady[Dados Prontos]
        CheckLocal -- Não --> DataReady
    end

    DataReady --> Enrich

    %% ETAPA 2: Enriquecimento
    subgraph "2. Enriquecimento (Yahoo Finance)"
        Enrich[🔍 Enrich Tickers with Names]
        Enrich -->|Busca nome oficial da empresa| TickersEnriched[/DataFrame Enriquecido/]
    end

    TickersEnriched --> MatchLoop

    %% ETAPA 3: Lógica de Matching (O Coração do Script)
    subgraph "3. Matching Otimizado (Iteração por Ticker)"
        MatchLoop[🔄 Loop: Para cada Ticker] --> SearchName{Tem Nome<br>do Yahoo?}
        
        SearchName -- Não --> UseTicker[Usar Ticker como Nome]
        SearchName -- Sim --> CleanName[Limpar Sufixos S.A./PN/ON]
        
        UseTicker --> Step1
        CleanName --> Step1

        %% Prioridade 1: Local
        Step1{1. Busca Local?} -->|Fuzzy Match em df_local| ScoreLocal{Score >= 70?}
        
        ScoreLocal -- Sim --> SetLocal[✅ Definir CNPJ Local]
        SetLocal --> SourceLocal[Source: local_fundamentals]
        
        ScoreLocal -- Não --> Step2

        %% Prioridade 2: CVM Geral
        Step2{2. Busca CVM?} -->|Fuzzy Match em df_cvm| ScoreCVM{Score CVM > Score Local?}
        
        ScoreCVM -- Não --> NoMatch[❌ Sem Match Confiável]
        NoMatch --> SourceNone[Source: none]

        ScoreCVM -- Sim --> SetCVM[✅ Definir CNPJ da CVM]
        
        %% CORREÇÃO AQUI: Aspas adicionadas para segurança
        SetCVM --> CheckCross{"CNPJ existe<br>no Local?"}
        
        CheckCross -- Sim --> SourceVer[Source: cvm_registry_verified]
        CheckCross -- Não --> SourceNew[⚠️ Source: cvm_registry_new]

        %% Saídas do Loop
        SourceLocal --> AppendRow
        SourceNone --> AppendRow
        SourceVer --> AppendRow
        SourceNew --> AppendRow
        
        AppendRow[Adicionar à Lista Final] --> NextTicker{Próximo?}
        NextTicker -- Sim --> MatchLoop
    end

    NextTicker -- Não --> End

In [ ]:
import pandas as pd
import requests
import base64
import json
import logging
import time
import urllib3
import io
import yfinance as yf
from pathlib import Path
from rapidfuzz import process, fuzz

# --- Configuração ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Diretórios
OUTPUT_DIR = Path("../data/dados_mapeamento")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

FUNDAMENTALS_PATH = Path("../data/cvm/final/fundamentals_wide.csv") 

URL_CVM_CADASTRO = "https://dados.cvm.gov.br/dados/CIA_ABERTA/CAD/DADOS/cad_cia_aberta.csv"

def fetch_ibrx100_from_b3_api() -> pd.DataFrame:
    """Busca tickers do IBRX-100 direto da API da B3."""
    logger.info("📡 [B3] Iniciando requisição à API...")
    
    try:
        params = {"language": "pt-br", "pageNumber": 1, "pageSize": 120, "index": "IBXX", "segment": "1"}
        params_b64 = base64.b64encode(json.dumps(params).encode("utf-8")).decode("utf-8")
        url = f"https://sistemaswebb3-listados.b3.com.br/indexProxy/indexCall/GetPortfolioDay/{params_b64}"
        
        urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
        response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, timeout=15, verify=False)
        
        if response.status_code != 200: return None
        
        data = response.json()
        results = data.get('results', [])
        
        if not results: return None
        
        df = pd.DataFrame(results)
        
        coluna_ticker = next((col for col in ['codNeg', 'cod', 'acronym', 'asset'] if col in df.columns), None)
        
        if not coluna_ticker:
            logger.error("❌ Coluna de ticker não encontrada no JSON da B3.")
            return None
            
        df_final = df[[coluna_ticker]].rename(columns={coluna_ticker: 'ticker'})
        df_final['ticker'] = df_final['ticker'].str.strip()
        df_final['ticker_yahoo'] = df_final['ticker'] + ".SA"
        
        logger.info(f"✅ [B3] {len(df_final)} ativos recuperados.")
        return df_final

    except Exception as e:
        logger.error(f"❌ Erro B3: {e}")
        return None

def fetch_cvm_registry() -> pd.DataFrame:
    """Baixa e processa o cadastro oficial de CNPJs da CVM."""
    logger.info("🏛️ [CVM] Baixando cadastro oficial de companhias...")
    
    try:
        response = requests.get(URL_CVM_CADASTRO, timeout=30)
        if response.status_code != 200:
            return None
            
        csv_content = io.StringIO(response.content.decode('latin1')) 
        df_cvm = pd.read_csv(csv_content, sep=';', dtype=str)
        
        df_cvm = df_cvm[df_cvm['SIT'] == 'ATIVO']
        
        df_cvm = df_cvm[['CNPJ_CIA', 'DENOM_SOCIAL']].copy()
        df_cvm['nome_limpo'] = df_cvm['DENOM_SOCIAL'].str.upper().str.strip()
        
        return df_cvm
    except Exception as e:
        logger.error(f"❌ Erro CVM: {e}")
        return None

def load_local_fundamentals() -> pd.DataFrame:
    """
    Carrega o arquivo local fundamentals_wide.csv para usar como 
    fonte prioritária de 'Match'.
    """
    logger.info(f"📂 [Local] Carregando dados fundamentais de: {FUNDAMENTALS_PATH}")
    
    if not FUNDAMENTALS_PATH.exists():
        logger.warning(f"⚠️ Arquivo local {FUNDAMENTALS_PATH} não encontrado. Otimização será ignorada.")
        return None
        
    try:
        df_fund = pd.read_csv(
            FUNDAMENTALS_PATH, 
            sep=';', 
            usecols=['CNPJ_CIA', 'DENOM_CIA'],
            encoding='utf-8-sig' # Ou 'latin1' dependendo de como você salvou
        )
        
        df_fund = df_fund.drop_duplicates(subset=['CNPJ_CIA']).copy()
        df_fund['nome_limpo'] = df_fund['DENOM_CIA'].str.upper().str.strip()
        
        logger.info(f"✅ [Local] {len(df_fund)} empresas únicas carregadas do histórico.")
        return df_fund
        
    except Exception as e:
        logger.error(f"❌ Erro ao ler arquivo local: {e}")
        return None

def enrich_tickers_with_names(df_b3: pd.DataFrame) -> pd.DataFrame:
    """Usa yfinance para descobrir o nome oficial da empresa por trás do ticker."""
    logger.info("🔍 [Enriquecimento] Buscando nomes das empresas via Yahoo Finance...")
    
    names_map = {}
    tickers_list = df_b3['ticker_yahoo'].tolist()
    
    total = len(tickers_list)
    
    tickers_obj = yf.Tickers(" ".join(tickers_list))
    
    for i, ticker in enumerate(tickers_list):
        try:
            info = tickers_obj.tickers[ticker].info
            name = info.get('longName') or info.get('shortName')
            names_map[ticker] = name.upper() if name else None
        except Exception:
            names_map[ticker] = None
        
        if i % 20 == 0:
            logger.info(f"   Processado {i}/{total}...")

    df_b3['nome_yahoo'] = df_b3['ticker_yahoo'].map(names_map)
    
    clean_names = df_b3['nome_yahoo'].str.replace(r'\s(PN|ON|UNIT|N1|N2|NM|S\.A\.|LTDA)$', '', regex=True)
    df_b3['nome_busca'] = clean_names.fillna(df_b3['ticker']) # Fallback para o ticker se não achar nome
    
    return df_b3

def match_ticker_cnpj_optimized(df_b3: pd.DataFrame, df_cvm: pd.DataFrame, df_local: pd.DataFrame = None) -> pd.DataFrame:
    """
    Cruza tickers com CNPJs usando uma estratégia em duas etapas:
    1. Prioridade: Busca no arquivo local (fundamentals_wide).
    2. Fallback: Busca no cadastro geral da CVM.
    """
    logger.info("🤝 [Matching] Cruzando bases com OTIMIZAÇÃO LOCAL...")
    
    matches = []
    
    local_names = []
    local_lookup = {}
    if df_local is not None:
        local_names = df_local['nome_limpo'].tolist()
        local_lookup = df_local.set_index('nome_limpo')['CNPJ_CIA'].to_dict()
    
    cvm_names = df_cvm['nome_limpo'].tolist()
    cvm_lookup = df_cvm.set_index('nome_limpo')['CNPJ_CIA'].to_dict()
    
    for _, row in df_b3.iterrows():
        ticker = row['ticker']
        search_name = row['nome_busca']
        
        if not search_name:
            matches.append({'ticker': ticker, 'CNPJ': None, 'match_score': 0, 'source': 'none'})
            continue

        best_name = None
        score = 0
        cnpj = None
        source = 'none'

        if local_names:
            match_local = process.extractOne(search_name, local_names, scorer=fuzz.token_sort_ratio)
            if match_local:
                name_l, score_l, _ = match_local
                if score_l >= 70: 
                    best_name = name_l
                    score = score_l
                    cnpj = local_lookup.get(best_name)
                    source = 'local_fundamentals'
        
        if score < 70:
            match_cvm = process.extractOne(search_name, cvm_names, scorer=fuzz.token_sort_ratio)
            if match_cvm:
                name_c, score_c, _ = match_cvm
                
                if score_c > score:
                    best_name = name_c
                    score = score_c
                    cnpj = cvm_lookup.get(best_name)
                    
                    in_local = cnpj in local_lookup.values()
                    source = 'cvm_registry_verified' if in_local else 'cvm_registry_new'

        matches.append({
            'ticker': ticker,
            'nome_b3_yahoo': search_name,
            'nome_oficial': best_name,
            'CNPJ': cnpj,
            'match_score': score,
            'source': source 
        })
            
    return pd.DataFrame(matches)

def main():
    start_time = time.time()
    
    df_b3 = fetch_ibrx100_from_b3_api()
    if df_b3 is None: return

    df_cvm = fetch_cvm_registry()
    if df_cvm is None: return

    df_local = load_local_fundamentals()

    df_b3_enriched = enrich_tickers_with_names(df_b3)

    df_final = match_ticker_cnpj_optimized(df_b3_enriched, df_cvm, df_local)

    final_path = OUTPUT_DIR / "mapa_ticker_cnpj_otimizado.csv"
    df_final.to_csv(final_path, index=False, sep=';', encoding='utf-8-sig')
    
    print("\n--- Resultado Final (Amostra) ---")
    try:
        display(df_final.head(15))
    except NameError:
        print(df_final.head(15).to_string())
    
    total = len(df_final)
    encontrados = df_final['CNPJ'].notna().sum()
    
    from_local = len(df_final[df_final['source'] == 'local_fundamentals'])
    from_cvm_ver = len(df_final[df_final['source'] == 'cvm_registry_verified'])
    from_cvm_new = len(df_final[df_final['source'] == 'cvm_registry_new'])

    print(f"\n📊 Estatísticas de Mapeamento:")
    print(f"   Total Tickers: {total}")
    print(f"   Mapeados: {encontrados} ({(encontrados/total)*100:.1f}%)")
    print(f"   --------------------------------")
    print(f"   ✅ Encontrado no Histórico Local: {from_local}")
    print(f"   ✅ Encontrado na CVM (Já existe no local): {from_cvm_ver}")
    print(f"   ⚠️ Encontrado na CVM (Novo/Sem dados locais): {from_cvm_new}")
    
    print(f"⏱️ Tempo total: {time.time() - start_time:.2f} segundos")

if __name__ == "__main__":
    main()